<a href="https://colab.research.google.com/github/jonG312/LangChain-/blob/main/langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Basic congfig
# Install basic package and set key
!pip install python-dotenv==1.0.0
!pip install langchain==0.0.137
!pip install pinecone-client==2.2.1
!pip install openai

In [ ]:
# Load environment variables
import os

os.environ["OPENAI_API_KEY"] = ''

In [ ]:

# Run basic query with OpenAI wrapper

from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
text = "What would be a good company name for a company that make colorful socks?"
print(llm(text))


In [ ]:
!pip install huggingface_hub

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [ ]:
from langchain import HuggingFaceHub

In [51]:
# https://huggingface.co/google/flan-t5-xl
llm = HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":0, "max_length":40})



2. Prompt Templates

In [ ]:
llm("Can Barack Obama have a conversation with George Washington?")

In [ ]:
prompt = """Question: Can Barack Obama have a conversation with George Washington?

Let's think step by step.

Answer: """
llm(prompt)

In [52]:
from langchain import PromptTemplate

template = """Question: {question}

Let's think step by step.

Answer: """

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
prompt.format(question="Can Barack Obama have a conversation with George Washington?")

In [ ]:
llm(prompt)

3. Chains:
Combine LLMs and Prompts in multi-step workflows

In [79]:
from langchain import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Can Barack Obama have a conversation with George Washington?"

print(llm_chain.run(question))

 No, Barack Obama cannot have a conversation with George Washington because George Washington is deceased.


4. Agents and Tools

Agents involve an LLM making decisions about which Actions to take, taking that Action, seeing an Observation, and repeating that until done.

When used correctly agents can be extremely powerful. In order to load agents, you should understand the following concepts:

Tool: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains.
LLM: The language model powering the agent.
Agent: The agent to use.

In [56]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [ ]:
!pip install wikipedia

In [61]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

In [72]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [ ]:
agent.run("In what year was the film Departed with Leopnardo Dicaprio released? What is this year raised to the 0.43 power?")

5. Memory




Add State to Chains and Agents.
Memory is the concept of persisting state between calls of a chain/agent. LangChain provides a standard interface for memory, a collection of memory implementations, and examples of chains/agents that use memory.

In [ ]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.predict(input="Hi there!")

In [ ]:
conversation.predict(input="Can we talk about AI?")

In [ ]:
conversation.predict(input="I'm interested in Reinforcement Learning.")

6. Document Loaders

Combining language models with your own text data is a powerful way to differentiate them. The first step in doing this is to load the data into “documents” - a fancy way of say some pieces of text. This module is aimed at making this easy.

In [66]:
from langchain.document_loaders import NotionDirectoryLoader

loader = NotionDirectoryLoader("Notion_DB")

docs = loader.load()

7. Indexes

Indexes refer to ways to structure documents so that LLMs can best interact with them. This module contains utility functions for working with documents

Embeddings: An embedding is a numerical representation of a piece of information, for example, text, documents, images, audio, etc.
Text Splitters: When you want to deal with long pieces of text, it is necessary to split up that text into chunks.
Vectorstores: Vector databases store and index vector embeddings from NLP models to understand the meaning and context of strings of text, sentences, and whole documents for more accurate and relevant search results.

In [67]:
import requests

url = "https://raw.githubusercontent.com/hwchase17/langchain/master/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
  f.write(res.text)

In [68]:
# Document Loader
from langchain.document_loaders import TextLoader
loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

In [69]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [ ]:
!pip install sentence_transformers

In [ ]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

#text = "This is a test document."
#query_result = embeddings.embed_query(text)
#doc_result = embeddings.embed_documents([text])

In [ ]:
!pip install faiss-cpu

In [74]:
# Vectorstore: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

In [ ]:
print(docs[0].page_content)

In [ ]:
db.save_local("faiss_index")
new_db = FAISS.load_local("faiss_index", embeddings)
docs = new_db.similarity_search(query)
print(docs[0].page_content)